## IMPORTACION DE DATOS DE CADA EVENTO

In [2]:
import requests as req
import pandas as pd
import numpy as np
import re
import statistics as stats

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
PATH = 'driver/chromedriver'

import tqdm
from bs4 import BeautifulSoup
import time
import re
import json

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../src')

In [3]:
options=Options()
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-allow-origins=*");
options.add_argument("user-data-dir=cookies")
#options.add_argument('--headless')                 #Habilitar si no queremos ver la ventana
options.add_experimental_option("detach", True)    #Esta opción corrige el error de cierre repentino
options.add_argument('--start-minimized')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--disable-extensions')
options.add_argument('--disable-infobars')

Tras importar las librerías necesarias y ejecutar las opciones, doy paso a retirar las cookies del driver que voy a utilizar. 
- Al habilitar la opcion que mantiene las cookies eliminadas, solamente ejecutamos el siguiente webdriver igualando options a options y ya nos mantendrá el driver sin las cookies.

#### 1. COMIENZO EL SCRAPING DE CADA UNO DE LOS EVENTOS

Tengo los links que ya he recogido a través de webscraping. 
- Importo el csv 'links' con el que voy a hacer la llamada a cada uno de los eventos (link a la compra del evento)

In [4]:
driver = webdriver.Chrome(options=options)

In [5]:
driver.get('https://www.entradas.com/city/madrid-370/?page=1')

In [6]:
links = pd.read_csv('../data/links.csv', encoding='utf-8')

In [7]:
links.head()

,Links
0,/artist/el-rey-leon/
1,/artist/circo-del-sol/alegria-cirque-du-soleil...
2,/artist/mago-pop/el-mago-pop-nada-es-imposible...
3,/artist/bruce-springsteen/
4,/artist/mamma-mia/


Genero una función que me servirá para ir iterando a través de todos los eventos, a través de su URL, con los links que me he generado en la variable 'links'.

- Pasaré por cada uno de las URL, con mi código, y me cogerá aquellos datos que son para mí interesantes: 
           - el nombre del evento o artista

- Hay un problema, y es que, aunque hay muchas url que no tienen más de una página, iteran sobre las 4 páginas, ya que el link le sigue permitiendo "pasar de página", aunque la información no cambia. 
       Por tanto, me funciona mucho mas lento, y esto me complica el scrapeo. 

- Genero en el propio código una linea más 'no_link', en la que, si encuentra un mensaje al inicio, al entrar en la nueva página, entonces salte al siguiente link (ya que el mensaje muestra en casi todas las ocasiones: "No hay más eventos para el artista")

In [8]:
def extract(link, pag):
    all_data = []  # Lista para almacenar los datos de todas las páginas
    
    try:
        for i in range(1, pag):  # Asumiendo que quieres iterar desde la página 1 hasta la página "pag"
            print("pag", i)
            driver.get(f"https://www.entradas.com{link}/?pnum={i}")

            no_link = driver.find_elements(By.XPATH, './/div/h2[@class="event-listing-headline"]')

            if no_link:
                print(f"No hay más eventos en la página {i} de {link}")
                break

            data = []
            print("0")

            # Encontrar todos los elementos de eventos en la página
            elements = driver.find_elements(By.XPATH, '//div[@class="u-position-relative js-component-loader js-load-event-list js-track-teaser-general js-track-general"]')
            print("1")

            for e in elements:
                print("2")
                event_name = e.find_elements(By.XPATH, './/div[@class="js-cal-listing-info-inner col-xs-9 col-sm-8 col-md-7 event-listing-info-inner"]/h2')
                eventos = [event.text for event in event_name]
                print("3")

                date_elements = e.find_elements(By.CSS_SELECTOR, 'time.event-listing-date')    
                fechas = [date_elements.get_attribute('datetime') for date_elements in date_elements]

                ubi_ele = e.find_elements(By.XPATH, './/div[@class="js-cal-listing-info-inner col-xs-9 col-sm-8 col-md-7 event-listing-info-inner"]/ul/li[2]')
                ubicacion = [ubi.text for ubi in ubi_ele]

                price = e.find_elements(By.XPATH, './/div/span[@class="event-listing-price event-price-available"]/span')
                precios = [precio.text for precio in price]

                enlace = e.find_elements(By.XPATH, './/div[@class="col-sm-4 col-md-3 event-listing-buy margin-right-xs hidden-xs js-event-listing-buy"]/a')
                enlaces = [enl.get_attribute('href') for enl in enlace]

                data.append([eventos, fechas, ubicacion, precios, enlaces])
                print("4")

            all_data.extend(data)  # Agregar los datos de la página actual a la lista

    except Exception as ex:
        print(f"Ocurrió un problema con {link}. Error: {str(ex)}")

    return all_data  # Devolver todos los datos recopilados de todas las páginas

- Comienzo con los primeros 7 links (primeras URL)
- A partir de ahí, voy a ir de pocos en pocos links. Algunos tienen muchos datos, ya que del mismo evento puede haber varias actuaciones (varias fechas u horas), mientras que otros solamente tendrán una fecha.
- Creo una función en la que me genera una lista con los datos de los eventos recogidos. Cada lista que voy sacando la guardo en un dataframe. Y estos posteriormente los uniré para crear un unico csv de mis datos importados.


## PRIMER DF (df0)

In [11]:
links.Links[0:7]

0                                 /artist/el-rey-leon/
1    /artist/circo-del-sol/alegria-cirque-du-soleil...
2    /artist/mago-pop/el-mago-pop-nada-es-imposible...
3                           /artist/bruce-springsteen/
4                                   /artist/mamma-mia/
5         /artist/ivan-ferreiro/ivan-ferreiro-3411430/
6                       /artist/carino/carino-3503691/
Name: Links, dtype: object

In [133]:
lst0=[]
for link in links.Links[0:7]:
    lst0.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
0
1
2
3
4
pag 4
No hay más eventos en la página 4 de /artist/el-rey-leon/
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
0
1
2
3
4
pag 4
No hay más eventos en la página 4 de /artist/circo-del-sol/alegria-cirque-du-soleil-madrid-3329203/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/mago-pop/el-mago-pop-nada-es-imposible-broadway-edition-madrid-3048612/
pag 1
No hay más eventos en la página 1 de /artist/bruce-springsteen/
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
0
1
2
3
4
pag 4
0
1
2
3
4
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/ivan-ferreiro/ivan-ferreiro-3411430/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/carino/carino-3503691/


In [135]:
from suport import df_event
df0 = df_event(lst0)

In [137]:
df0 = pd.DataFrame(df0)

In [142]:
df0.to_csv('df0.csv', index=False, encoding='utf-8')

Voy cambiando de rangos, en los links, para ir llevando un control de aquellos que voy ejecutando. 

## SIGUIENTE (df1)

In [144]:
links.Links[7:10]

7                                      /artist/trashi/
8    /artist/circo-del-sol/alegria-cirque-du-soleil...
9    /artist/cantajuego/cantajuego-gira-2023-2024-3...
Name: Links, dtype: object

In [145]:
lst=[]
for link in links.Links[7:10]:
    lst.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/trashi/
pag 1
0
1
pag 2
0
1
pag 3
0
1
pag 4
0
1
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/cantajuego/cantajuego-gira-2023-2024-3475836/


In [147]:
from suport import df_event
df1 = df_event(lst)

In [148]:
df1 = pd.DataFrame(df1)

In [149]:
df1.to_csv('df1.csv', index=False, encoding='utf-8')

## SIGUIENTE (df2)

In [ ]:
links.Links[10:15]

In [ ]:
lst1=[]
for link in links.Links[10:15]:
    lst1.append(extract(link, 5))

In [ ]:
lst1

In [ ]:
# Lista para almacenar los datos
data = []

# Iterar sobre la lista principal
for eventos in lst1:
    # Iterar sobre cada sublista de eventos
    for evento in eventos:
        # Desglosar la información del evento
        for i in range(len(evento[0])):
            nombre = evento[0][i]
            fecha = evento[1][i]
            ubicacion = evento[2][i]
            precio = evento[3][i]
            enlace = evento[4][i]

            # Crear un diccionario con la información del evento
            evento_dict = {
                'Nombre': nombre,
                'Fecha': fecha,
                'Ubicacion': ubicacion,
                'Precio': precio,
                'Enlace': enlace
            }

            # Agregar el diccionario a la lista de datos
            data.append(evento_dict)

# Crear un DataFrame a partir de la lista de datos
df2 = pd.DataFrame(data)

# Mostrar el DataFrame

In [ ]:
df2.to_csv('df2.csv', index=False, encoding='utf-8')

## SIGUIENTE (df3)

In [ ]:
links.Links[15:20]

In [ ]:
lst2=[]
for link in links.Links[15:20]:
    lst2.append(extract(link, 5))

In [ ]:
lst2

In [ ]:
# Lista para almacenar los datos
data = []

# Iterar sobre la lista principal
for eventos in lst2:
    # Iterar sobre cada sublista de eventos
    for evento in eventos:
        # Desglosar la información del evento
        for i in range(len(evento[0])):
            nombre = evento[0][i]
            fecha = evento[1][i]
            ubicacion = evento[2][i]
            precio = evento[3][i]
            enlace = evento[4][i]

            # Crear un diccionario con la información del evento
            evento_dict = {
                'Nombre': nombre,
                'Fecha': fecha,
                'Ubicacion': ubicacion,
                'Precio': precio,
                'Enlace': enlace
            }

            # Agregar el diccionario a la lista de datos
            data.append(evento_dict)

# Crear un DataFrame a partir de la lista de datos
df3 = pd.DataFrame(data)

# Mostrar el DataFrame

In [ ]:
df3.to_csv('df3.csv', index=False, encoding='utf-8')

In [ ]:
#driver.quit()

## SIGUIENTE (df4)

In [10]:
links.Links[20:25]

20                  /artist/jamming-show/
21                        /artist/belako/
22           /artist/exhibicion-en-vuelo/
23         /artist/la-madre-que-me-pario/
24    /artist/en-ocasiones-veo-a-umberto/
Name: Links, dtype: object

In [11]:
lst3=[]
for link in links.Links[20:25]:
    lst3.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/jamming-show/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/belako/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/exhibicion-en-vuelo/
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
0
1
2
3
4
pag 4
No hay más eventos en la página 4 de /artist/la-madre-que-me-pario/
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
No hay más eventos en la página 3 de /artist/en-ocasiones-veo-a-umberto/


In [13]:
# Lista para almacenar los datos
data = []

# Iterar sobre la lista principal
for eventos in lst3:
    # Iterar sobre cada sublista de eventos
    for evento in eventos:
        # Desglosar la información del evento
        for i in range(len(evento[0])):
            nombre = evento[0][i]
            fecha = evento[1][i]
            ubicacion = evento[2][i]
            precio = evento[3][i]
            enlace = evento[4][i]

            # Crear un diccionario con la información del evento
            evento_dict = {
                'Nombre': nombre,
                'Fecha': fecha,
                'Ubicacion': ubicacion,
                'Precio': precio,
                'Enlace': enlace
            }

            # Agregar el diccionario a la lista de datos
            data.append(evento_dict)

# Crear un DataFrame a partir de la lista de datos
df4 = pd.DataFrame(data)

In [15]:
df4.to_csv('df4.csv', index=False, encoding='utf-8')

## SIGUIENTE (df5)

In [ ]:
links.Links[25:30]

In [16]:
lst4=[]
for link in links.Links[25:30]:
    lst4.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/real-madrid-c-f/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/biznaga/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/mayumana-/mayumana-impulso-madrid-3486653/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/circlassica/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/pica-pica/


In [18]:
# Lista para almacenar los datos
data = []

# Iterar sobre la lista principal
for eventos in lst4:
    # Iterar sobre cada sublista de eventos
    for evento in eventos:
        # Desglosar la información del evento
        for i in range(len(evento[0])):
            nombre = evento[0][i]
            fecha = evento[1][i]
            ubicacion = evento[2][i]
            precio = evento[3][i]
            enlace = evento[4][i]

            # Crear un diccionario con la información del evento
            evento_dict = {
                'Nombre': nombre,
                'Fecha': fecha,
                'Ubicacion': ubicacion,
                'Precio': precio,
                'Enlace': enlace
            }

            # Agregar el diccionario a la lista de datos
            data.append(evento_dict)

# Crear un DataFrame a partir de la lista de datos
df5 = pd.DataFrame(data)

In [20]:
df5.to_csv('df5.csv', index=False, encoding='utf-8')

## SIGUIENTE (df6)

In [21]:
links.Links[30:35]

30                     /artist/el-club-de-la-comedia/
31    /artist/de-caperucita-a-loba-en-solo-seis-tios/
32                      /artist/eugenio-solo-hay-uno/
33                                    /artist/morreo/
34                           /artist/faemino-cansado/
Name: Links, dtype: object

In [22]:
lst5=[]
for link in links.Links[30:35]:
    lst5.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/el-club-de-la-comedia/
pag 1
No hay más eventos en la página 1 de /artist/de-caperucita-a-loba-en-solo-seis-tios/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/eugenio-solo-hay-uno/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/morreo/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/faemino-cansado/


In [25]:
from suport import df_event
df6 = df_event(lst5)
df6 = pd.DataFrame(df6)

In [26]:
df6

,Nombre,Fecha,Ubicacion,Precio,Enlace
0,Feliz Año Neox presenta El Club de la Comedia,2023-12-13T21:00:00.000+01:00,Teatro Diéguez,"desde 5,00 €",https://www.entradas.com/event/feliz-ano-neox-...
1,Eugenio solo hay uno,2023-12-11T20:30:00.000+01:00,Teatro Coliseum,"desde 20,00 €",https://www.entradas.com/event/pablo-chiapella...
2,Morreo - Alegría Club!,2024-01-18T21:30:00.000+01:00,Sala El Sol,"desde 14,50 €",https://www.entradas.com/event/morreo-alegria-...
3,Faemino y Cansado - 17 veces - Madrid,2023-12-04T20:00:00.000+01:00,Sala Capitol,"desde 22,00 €",https://www.entradas.com/event/faemino-y-cansa...
4,Faemino y Cansado - 17 veces - Madrid,2024-01-15T20:00:00.000+01:00,Sala Capitol,"desde 24,00 €",https://www.entradas.com/event/faemino-y-cansa...


In [59]:
df6.to_csv('df6.csv', index=False, encoding='utf-8')

## SIGUIENTE (df7)

In [30]:
links.Links[35:40]

35        /artist/jorge-blass/
36      /artist/miguel-poveda/
37    /artist/luis-piedrahita/
38    /artist/diego-el-cigala/
39        /artist/sexy-zebras/
Name: Links, dtype: object

In [ ]:
lst6=[]
for link in links.Links[35:40]:
    lst6.append(extract(link, 5))

In [33]:
from suport import df_event
df7 = df_event(lst6)

In [60]:
df7 = pd.DataFrame(df7)
df7.to_csv('df7.csv', index=False, encoding='utf-8')

In [37]:
#driver.quit()

## SIGUIENTE (df8)

In [43]:
links.Links[40:45]

40    /artist/symphonic-rhapsody-of-queen/symphonic-...
41                         /artist/ilustres-ignorantes/
42                                /artist/rafa-sanchez/
43                      /artist/real-madrid-baloncesto/
44                                         /artist/obk/
Name: Links, dtype: object

In [44]:
lst7=[]
for link in links.Links[40:45]:
    lst7.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/symphonic-rhapsody-of-queen/symphonic-rhapsody-of-queen-3295553/
pag 1
No hay más eventos en la página 1 de /artist/ilustres-ignorantes/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/rafa-sanchez/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/real-madrid-baloncesto/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/obk/


In [ ]:
from suport import df_event
df8 = df_event(lst7)

In [61]:
df8 = pd.DataFrame(df8)
df8.to_csv('df8.csv', index=False, encoding='utf-8')

## SIGUIENTE (df9)

In [50]:
links.Links[45:50]

45    /artist/rock-en-familia/rock-en-familia-hijos-...
46    /artist/rock-en-familia/rock-en-familia-i-love...
47    /artist/rock-en-familia/rock-en-familia-i-love...
48    /artist/rock-en-familia/rock-en-familia-queenm...
49                           /artist/peter-pan-musical/
Name: Links, dtype: object

In [51]:
lst8=[]
for link in links.Links[45:50]:
    lst8.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/rock-en-familia/rock-en-familia-hijos-del-rock-and-roll-3272770/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/rock-en-familia/rock-en-familia-i-love-rock-and-roll-3221042/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/rock-en-familia/rock-en-familia-i-love-heavy-metal-3449495/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/rock-en-familia/rock-en-familia-queenmania-3294746/
pag 1
No hay más eventos en la página 1 de /artist/peter-pan-musical/


In [153]:
from suport import df_event
df9 = df_event(lst8)

In [157]:
df9 = pd.DataFrame(df9)
df9.to_csv('df9.csv', index=False, encoding='utf-8')

## SIGUIENTE (df10)

In [54]:
links.Links[50:55]

50                /artist/morgan/morgan-madrid-3005721/
51                                  /artist/sex-museum/
52                                /artist/alex-clavero/
53                                 /artist/david-guapo/
54    /artist/atletico-de-madrid/territorio-atleti-m...
Name: Links, dtype: object

In [62]:
lst9=[]
for link in links.Links[50:55]:
    lst9.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/morgan/morgan-madrid-3005721/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/sex-museum/
pag 1
No hay más eventos en la página 1 de /artist/alex-clavero/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/david-guapo/
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
0
1
2
3
4
pag 4
0
1
2
3
4


In [158]:
from suport import df_event
df10 = df_event(lst9)

In [160]:
df10 = pd.DataFrame(df10)
df10.to_csv('df10.csv', index=False, encoding='utf-8')

## SIGUIENTE (df11)

In [76]:
links.Links[55:65]

55                                /artist/los-zigarros/
56                          /artist/aladdin-el-musical/
57                        /artist/el-sentido-del-humor/
58                               /artist/queralt-lahoz/
59    /artist/el-brujo/el-brujo-los-dioses-y-dios-ma...
60    /artist/el-brujo/el-brujo-el-lazarillo-de-torm...
61    /artist/el-brujo/el-brujo-el-viaje-del-monstru...
62    /artist/el-brujo/el-brujo-autobiografia-de-un-...
63                             /artist/alex-o-dogherty/
64       /artist/india-martinez/india-martinez-3473270/
Name: Links, dtype: object

In [70]:
lst10=[]
for link in links.Links[55:65]:
    lst10.append(extract(link, 5))

pag 1
No hay más eventos en la página 1 de /artist/los-zigarros/
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
0
1
2
3
4
pag 4
0
1
2
3
4
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/el-sentido-del-humor/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/queralt-lahoz/
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
No hay más eventos en la página 3 de /artist/el-brujo/el-brujo-los-dioses-y-dios-madrid-3038073/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/el-brujo/el-brujo-el-lazarillo-de-tormes-madrid-3527317/
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
No hay más eventos en la página 3 de /artist/el-brujo/el-brujo-el-viaje-del-monstruo-fiero-madrid-3534266/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/el-brujo/el-brujo-autobiografia-de-un-yogui-madrid-3527318/
pag 1
No hay más eventos en la página 1 de /artist/alex-o-dogherty/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/india-martinez/india-martinez-3473

In [162]:
from suport import df_event
df11 = df_event(lst10)

In [163]:
df11 = pd.DataFrame(df11)
df11.to_csv('df11.csv', index=False, encoding='utf-8')

## SIGUIENTE (df12)

In [77]:
links.Links[65:75]

65                                    /artist/malinche/
66                     /artist/el-fantasma-de-la-opera/
67    /artist/corta-el-cable-rojo/corta-el-cable-roj...
68    /artist/harlem-globetrotters/harlem-globetrott...
69                               /artist/el-monaguillo/
70                                     /artist/ternura/
71             /artist/la-bella-y-la-bestia-el-musical/
72                                         /artist/noa/
73    /artist/alicia-en-el-pais-de-las-maravillas-la...
74                      /artist/queen-we-will-rock-you/
Name: Links, dtype: object

In [79]:
lst11=[]
for link in links.Links[65:75]:
    lst11.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
0
1
2
3
4
pag 4
0
1
2
3
4
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
0
1
2
3
4
pag 4
No hay más eventos en la página 4 de /artist/el-fantasma-de-la-opera/
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
0
1
2
3
4
pag 4
No hay más eventos en la página 4 de /artist/corta-el-cable-rojo/corta-el-cable-rojo-madrid-1467602/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/harlem-globetrotters/harlem-globetrotters-madrid-3341839/
pag 1
No hay más eventos en la página 1 de /artist/el-monaguillo/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/ternura/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/la-bella-y-la-bestia-el-musical/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/noa/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/alicia-en-el-pais-de-las-maravillas-la-maquineta/
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
0
1
2
3
4
pag 4
0
1
2
3
4


In [164]:
from suport import df_event
df12 = df_event(lst11)

In [165]:
df12 = pd.DataFrame(df12)
df12.to_csv('df12.csv', index=False, encoding='utf-8')

In [84]:
#driver.quit()

## SIGUIENTE (df13)

In [91]:
links.Links[75:80]

75               /artist/whitney-houston-hologram-tour/
76       /artist/javi-rufo/123-magia-javi-rufo-2791317/
77    /artist/javi-rufo/magia-en-juego-javi-rufo-345...
78        /artist/javi-rufo/vinculos-javi-rufo-3539517/
79            /artist/la-granja-de-zenon-el-gran-gallo/
Name: Links, dtype: object

In [93]:
lst12=[]
for link in links.Links[75:80]:
    lst12.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/whitney-houston-hologram-tour/
pag 1
0
1
2
3
4
pag 2
0
1
2
3
4
pag 3
0
1
2
3
4
pag 4
No hay más eventos en la página 4 de /artist/javi-rufo/123-magia-javi-rufo-2791317/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/javi-rufo/magia-en-juego-javi-rufo-3459459/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/javi-rufo/vinculos-javi-rufo-3539517/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/la-granja-de-zenon-el-gran-gallo/


In [167]:
from suport import df_event
df13 = df_event(lst12)

In [169]:
df13 = pd.DataFrame(df13)
df13.to_csv('df13.csv', index=False, encoding='utf-8')

## SIGUIENTE (df14)

In [101]:
links.Links[80:85]

80    /artist/disney-on-ice/disney-on-ice-100-anos-d...
81           /artist/la-flauta-magica-mi-primera-opera/
82    /artist/el-mago-yunke/el-mago-yunke-madrid-321...
83    /artist/mocedades/mocedades-avanzando-hacia-el...
84                                 /artist/tito-nieves/
Name: Links, dtype: object

In [102]:
lst13=[]
for link in links.Links[80:85]:
    lst13.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/disney-on-ice/disney-on-ice-100-anos-de-emocion-en-madrid-3231572/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/la-flauta-magica-mi-primera-opera/
pag 1
No hay más eventos en la página 1 de /artist/el-mago-yunke/el-mago-yunke-madrid-3213684/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/mocedades/mocedades-avanzando-hacia-el-futuro-3411037/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/tito-nieves/


In [109]:
from suport import df_event
df14 = df_event(lst13)

In [112]:
df14 = pd.DataFrame(df14)
df14

,Nombre,Fecha,Ubicacion,Precio,Enlace
0,Disney On Ice Madrid 2024,2024-02-08T19:00:00.000+01:00,WiZink Center,"desde 16,00 €",https://www.entradas.com/event/disney-on-ice-1...
1,Disney On Ice Madrid 2024,2024-02-09T19:45:00.000+01:00,WiZink Center,,https://www.entradas.com/event/disney-on-ice-1...
2,Disney On Ice Madrid 2024,2024-02-10T12:00:00.000+01:00,WiZink Center,"desde 16,00 €",https://www.entradas.com/event/disney-on-ice-1...
3,Disney On Ice Madrid 2024,2024-02-10T16:00:00.000+01:00,WiZink Center,,https://www.entradas.com/event/disney-on-ice-1...
4,Disney On Ice Madrid 2024,2024-02-10T19:45:00.000+01:00,WiZink Center,"desde 15,00 €",https://www.entradas.com/event/disney-on-ice-1...
5,Disney On Ice Madrid 2024,2024-02-11T12:00:00.000+01:00,WiZink Center,,https://www.entradas.com/event/disney-on-ice-1...
6,Disney On Ice Madrid 2024,2024-02-11T16:00:00.000+01:00,WiZink Center,"desde 16,00 €",https://www.entradas.com/event/disney-on-ice-1...
7,La Flauta Mágica. Tu primera ópera,2023-12-17T12:00:00.000+01:00,Teatro Bellas Artes,"desde 8,00 €",https://www.entradas.com/event/la-flauta-magic...
8,La Flauta Mágica. Tu Primera Ópera,2024-01-14T12:00:00.000+01:00,Teatro Bellas Artes,"desde 8,00 €",https://www.entradas.com/event/la-flauta-magic...
9,La Flauta Mágica. Tu Primera Ópera,2024-01-21T12:00:00.000+01:00,Teatro Bellas Artes,"desde 8,00 €",https://www.entradas.com/event/la-flauta-magic...


In [170]:
df14.to_csv('df14.csv', index=False, encoding='utf-8')

## SIGUIENTE (df15)

In [115]:
links.Links[85:90]

85                 /artist/inverfest/rambalaya-3538755/
86    /artist/inverfest/pablo-und-destruktion-y-la-t...
87              /artist/inverfest/los-blenders-1618827/
88              /artist/inverfest/lucas-colman-3524523/
89                   /artist/inverfest/mr-zoom-3538756/
Name: Links, dtype: object

In [116]:
lst14=[]
for link in links.Links[85:90]:
    lst14.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/rambalaya-3538755/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/pablo-und-destruktion-y-la-tribu-del-trueno-3524538/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/los-blenders-1618827/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/lucas-colman-3524523/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/mr-zoom-3538756/


In [117]:
from suport import df_event
df15 = df_event(lst14)

In [119]:
df15 = pd.DataFrame(df15)

In [121]:
df15.to_csv('df15.csv', index=False, encoding='utf-8')

## SIGUIENTE (df16)

In [123]:
links.Links[90:100]

90    /artist/inverfest/txarly-usher-y-los-ejemplare...
91             /artist/inverfest/rosin-de-palo-3538754/
92        /artist/inverfest/kruder-dorfmeister-1730320/
93    /artist/inverfest/albert-pla-and-the-surprise-...
94               /artist/inverfest/bestia-bebe-3077633/
95    /artist/inverfest/destripando-la-historia-3526...
96              /artist/inverfest/grande-amore-3503693/
97    /artist/inverfest/theo-lawrence-the-possums-35...
98    /artist/inverfest/maximiliano-calvo-el-gallo-3...
99              /artist/inverfest/ricardo-moya-3503694/
Name: Links, dtype: object

In [124]:
lst15=[]
for link in links.Links[90:100]:
    lst15.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/txarly-usher-y-los-ejemplares-kinki-boys-3538729/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/rosin-de-palo-3538754/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/kruder-dorfmeister-1730320/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/albert-pla-and-the-surprise-band-2909144/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/bestia-bebe-3077633/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/destripando-la-historia-3526220/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/grande-amore-3503693/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/theo-lawrence-the-possums-3526227/
pag 1
No hay más eventos en la página 1 de /artist/inverfest/maximiliano-calvo-el-gallo-3503683/
pag 1
0
1
2
3
4
pag 2
No hay más 

In [126]:
from suport import df_event
df16 = df_event(lst15)

In [127]:
df16 = pd.DataFrame(df16)

In [129]:
df16.to_csv('df16.csv', index=False, encoding='utf-8')

In [171]:
#driver.quit()

## SIGUIENTE (df17)

In [175]:
links.Links[100:105]

100    /artist/inverfest/featherweight-oslo-ovnies-27...
101                   /artist/inverfest/mcenroe-3503685/
102                /artist/inverfest/paul-alone-3504465/
103    /artist/inverfest/the-magnetics-begona-bang-ma...
104    /artist/inverfest/muchachito-bombo-infierno-32...
Name: Links, dtype: object

In [174]:
lst16=[]
for link in links.Links[100:105]:
    lst16.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/featherweight-oslo-ovnies-2776391/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/mcenroe-3503685/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/paul-alone-3504465/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/the-magnetics-begona-bang-matu-3538750/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/muchachito-bombo-infierno-3225873/


In [176]:
from suport import df_event
df17 = df_event(lst16)

In [178]:
df17 = pd.DataFrame(df17)

In [180]:
df17.to_csv('./data/dfs_entradas/df17.csv', index=False, encoding='utf-8')

## SIGUIENTE (df18)

In [181]:
links.Links[105:110]

105                      /artist/inverfest/84-3538751/
106                /artist/inverfest/the-oxys-3538728/
107                  /artist/inverfest/tarque-3501745/
108    /artist/inverfest/christina-rosenvinge-2831739/
109           /artist/inverfest/fernandocosta-3526219/
Name: Links, dtype: object

In [182]:
lst17=[]
for link in links.Links[105:110]:
    lst17.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/84-3538751/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/the-oxys-3538728/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/tarque-3501745/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/christina-rosenvinge-2831739/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/fernandocosta-3526219/


In [183]:
from suport import df_event
df18 = df_event(lst17)

In [184]:
df18 = pd.DataFrame(df18)

In [185]:
df18.to_csv('./data/dfs_entradas/df18.csv', index=False, encoding='utf-8')

In [186]:
driver.quit()

## SIGUIENTE (df19)

In [13]:
links.Links[110:115]

110       /artist/inverfest/ruben-pozo-3506022/
111        /artist/inverfest/leo-rizzi-1687904/
112    /artist/inverfest/viva-belgrado-2907639/
113           /artist/inverfest/delgao-3504461/
114            /artist/inverfest/claim-3524543/
Name: Links, dtype: object

In [14]:
lst18=[]
for link in links.Links[100:105]:
    lst18.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/featherweight-oslo-ovnies-2776391/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/mcenroe-3503685/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/paul-alone-3504465/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/the-magnetics-begona-bang-matu-3538750/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/muchachito-bombo-infierno-3225873/


In [ ]:
from suport import df_event
df19 = df_event(lst18)

In [39]:
df19.to_csv('../data/dfs_entradas/df19.csv', index=False, encoding='utf-8')

## SIGUIENTE (df20)

In [44]:
links.Links[115:120]

115    /artist/inverfest/oddliquor-blindao-3220381/
116           /artist/inverfest/carmen-113-2908653/
117               /artist/inverfest/minino-3538749/
118        /artist/inverfest/pedro-ladroga-3524547/
119               /artist/inverfest/calizo-3538727/
Name: Links, dtype: object

In [45]:
lst19=[]
for link in links.Links[115:120]:
    lst19.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/oddliquor-blindao-3220381/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/carmen-113-2908653/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/minino-3538749/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/pedro-ladroga-3524547/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/calizo-3538727/


In [ ]:
from suport import df_event
df20 = df_event(lst19)

In [47]:
# Lista para almacenar los datos
data = []

# Iterar sobre la lista principal
for eventos in lst19:
    # Iterar sobre cada sublista de eventos
    for evento in eventos:
        # Desglosar la información del evento
        for i in range(len(evento[0])):
            nombre = evento[0][i]
            fecha = evento[1][i]
            ubicacion = evento[2][i]
            precio = evento[3][i]
            enlace = evento[4][i]

            # Crear un diccionario con la información del evento
            evento_dict = {
                'Nombre': nombre,
                'Fecha': fecha,
                'Ubicacion': ubicacion,
                'Precio': precio,
                'Enlace': enlace
            }

            # Agregar el diccionario a la lista de datos
            data.append(evento_dict)

# Crear un DataFrame a partir de la lista de datos
df20 = pd.DataFrame(data)

In [48]:
df20.to_csv('../data/dfs_entradas/df20.csv', index=False, encoding='utf-8')

## SIGUIENTE (df21)

In [49]:
links.Links[120:125]

120               /artist/inverfest/los-alvarez-3538747/
121                 /artist/inverfest/egon-soda-3524542/
122                 /artist/inverfest/austin-tv-3491782/
123    /artist/inverfest/ariel-rot-25-anos-hablando-s...
124           /artist/inverfest/israel-b-eed1hs-3361884/
Name: Links, dtype: object

In [50]:
lst20=[]
for link in links.Links[120:125]:
    lst20.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/los-alvarez-3538747/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/egon-soda-3524542/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/austin-tv-3491782/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/ariel-rot-25-anos-hablando-solo-3504415/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/israel-b-eed1hs-3361884/


In [ ]:
from suport import df_event
df21 = df_event(lst20)

In [52]:
df21.to_csv('../data/dfs_entradas/df21.csv', index=False, encoding='utf-8')

## SIGUIENTE (df22)

In [53]:
links.Links[125:130]

125          /artist/inverfest/yoly-saa-1553966/
126    /artist/inverfest/ninos-luchando-1612915/
127         /artist/inverfest/tu-suerte-3153569/
128      /artist/inverfest/chata-flores-2897145/
129            /artist/inverfest/mavica-1842950/
Name: Links, dtype: object

In [54]:
lst21=[]
for link in links.Links[125:130]:
    lst21.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/yoly-saa-1553966/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/ninos-luchando-1612915/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/tu-suerte-3153569/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/chata-flores-2897145/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/mavica-1842950/


In [ ]:
from suport import df_event
df22 = df_event(lst21)

In [55]:
# Lista para almacenar los datos
data = []

# Iterar sobre la lista principal
for eventos in lst21:
    # Iterar sobre cada sublista de eventos
    for evento in eventos:
        # Desglosar la información del evento
        for i in range(len(evento[0])):
            nombre = evento[0][i]
            fecha = evento[1][i]
            ubicacion = evento[2][i]
            precio = evento[3][i]
            enlace = evento[4][i]

            # Crear un diccionario con la información del evento
            evento_dict = {
                'Nombre': nombre,
                'Fecha': fecha,
                'Ubicacion': ubicacion,
                'Precio': precio,
                'Enlace': enlace
            }

            # Agregar el diccionario a la lista de datos
            data.append(evento_dict)

# Crear un DataFrame a partir de la lista de datos
df22 = pd.DataFrame(data)

In [56]:
df22.to_csv('../data/dfs_entradas/df22.csv', index=False, encoding='utf-8')

In [57]:
#driver.quit()

## SIGUIENTE (df23)

In [9]:
links.Links[130:135]

130      /artist/inverfest/flecha-valona-2638503/
131         /artist/inverfest/julia-amor-3083231/
132    /artist/inverfest/pinpilinpussies-3056767/
133             /artist/inverfest/bernal-3060541/
134                         /artist/maria-callas/
Name: Links, dtype: object

In [10]:
lst22=[]
for link in links.Links[130:135]:
    lst22.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/flecha-valona-2638503/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/julia-amor-3083231/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/pinpilinpussies-3056767/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/inverfest/bernal-3060541/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/maria-callas/


In [ ]:
from suport import df_event
df23 = df_event(lst22)

In [16]:
df23.to_csv('../data/dfs_entradas/df23.csv', index=False, encoding='utf-8')

## SIGUIENTE (df24)

In [18]:
links.Links[135:140]

135    /artist/villanos-del-jazz/antonio-lizana-2840178/
136     /artist/villanos-del-jazz/scary-pockets-3490507/
137                            /artist/love-yarn-madrid/
138    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
139    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
Name: Links, dtype: object

In [19]:
lst23=[]
for link in links.Links[135:140]:
    lst23.append(extract(link, 5))

pag 1
0
1
pag 2
0
1
pag 3
0
1
pag 4
0
1
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/villanos-del-jazz/scary-pockets-3490507/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/love-yarn-madrid/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-b16-jueves-3479308/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-b16-viernes-3479309/


In [ ]:
from suport import df_event
df24 = df_event(lst23)

In [21]:
df24.to_csv('../data/dfs_entradas/df24.csv', index=False, encoding='utf-8')

## SIGUIENTE (df25)

In [22]:
links.Links[140:145]

140    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
141    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
142    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
143    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
144    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
Name: Links, dtype: object

In [23]:
lst24=[]
for link in links.Links[140:145]:
    lst24.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-b18-jueves-3479312/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-a15-viernes-3479307/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-a19-viernes-3479315/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-a15-jueves-3479306/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-b18-viernes-3479313/


In [ ]:
from suport import df_event
df25 = df_event(lst24)

In [25]:
df25.to_csv('../data/dfs_entradas/df25.csv', index=False, encoding='utf-8')

## SIGUIENTE (df26)

In [26]:
links.Links[145:150]

145    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
146    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
147    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
148    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
149    /artist/orquesta-sinfonica-y-coro-de-rtve/conc...
Name: Links, dtype: object

In [27]:
lst25=[]
for link in links.Links[145:150]:
    lst25.append(extract(link, 5))

pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-b12-viernes-3479301/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-a13-jueves-3479302/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-a17-viernes-3479311/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-b14-jueves-3479304/
pag 1
0
1
2
3
4
pag 2
No hay más eventos en la página 2 de /artist/orquesta-sinfonica-y-coro-de-rtve/concierto-sinfonico-de-abono-b8-jueves-3479292/


In [ ]:
from suport import df_event
df26 = df_event(lst25)

In [31]:
df26.to_csv('../data/dfs_entradas/df26.csv', index=False, encoding='utf-8')

In [ ]:
#driver.quit()